## Load libraries

In [1]:
from gensim.models import KeyedVectors
import faiss
import numpy as np

In [2]:
NEIGHBORS = 50

word = 'проказа'

In [3]:
word_vectors_fpath = "models/wikipedia-ru-2018.word_vectors"

In [4]:
wv = KeyedVectors.load_word2vec_format(word_vectors_fpath, binary=False, unicode_errors="ignore")

/home/denis/Programs/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## knn by w2v

In [5]:
def get_nns(target, topn=NEIGHBORS):
    """
    target - word token
    topn - how many neighbors
    """
    nns = wv.most_similar(positive=[target], negative=[], topn=topn)
    #nns = [(word, score) for word, score in nns if minimize(word) != minimize(target)]
    return nns

In [14]:
%%time
get_nns(word)

CPU times: user 277 ms, sys: 4.04 ms, total: 281 ms
Wall time: 512 ms


[('малярия', 0.7535935640335083),
 ('гонорея', 0.7530273199081421),
 ('брюшной_тиф', 0.7527637481689453),
 ('хроническая', 0.7498410940170288),
 ('энцефалопатия', 0.7485659122467041),
 ('чесотка', 0.7483586072921753),
 ('волчанка', 0.7467740774154663),
 ('сибирская_язва', 0.744862973690033),
 ('пневмония', 0.7420924305915833),
 ('экзема', 0.7396251559257507),
 ('акатизия', 0.7387583255767822),
 ('диарея', 0.7369372844696045),
 ('панкреатит', 0.7364274263381958),
 ('ревматоидный_артрит', 0.7353441715240479),
 ('идиопатическая', 0.7351337671279907),
 ('кожный_зуд', 0.7348761558532715),
 ('скарлатина', 0.7332731485366821),
 ('энтерит', 0.7330397963523865),
 ('нейродермит', 0.7294857501983643),
 ('сифилис', 0.7289275527000427),
 ('корь', 0.7280865907669067),
 ('простатит', 0.7264933586120605),
 ('вирусная_инфекция', 0.7261444926261902),
 ('конъюнктивит', 0.7260156869888306),
 ('гангрена', 0.7253180742263794),
 ('себорея', 0.7248758673667908),
 ('склеродермия', 0.7244752645492554),
 ('сарко

## KNN by faiss

In [10]:
index = faiss.IndexFlatIP(wv.vector_size)
index.add(wv.syn0norm)

/home/denis/Programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [12]:
def get_nns_faiss(target, topn=NEIGHBORS):
    """
    Get target neighbors by faiss
    """
    nns_list = list()

    numpy_vec = np.array([wv[target]])
    D, I = index.search(numpy_vec, topn + 1)

    for _D, _I in zip(D, I):
        for n, (d, i) in enumerate(zip(_D.ravel(), _I.ravel())):
            if n > 0:
                nns_list.append((wv.index2word[i], d))

    return nns_list

In [15]:
%%time
get_nns_faiss(word)

CPU times: user 1.18 s, sys: 59.8 ms, total: 1.24 s
Wall time: 354 ms


[('малярия', 1.9416015),
 ('гонорея', 1.9401428),
 ('брюшной_тиф', 1.9394639),
 ('хроническая', 1.9319334),
 ('энцефалопатия', 1.9286476),
 ('чесотка', 1.9281138),
 ('волчанка', 1.924032),
 ('сибирская_язва', 1.9191079),
 ('пневмония', 1.9119697),
 ('экзема', 1.9056126),
 ('акатизия', 1.9033797),
 ('диарея', 1.8986872),
 ('панкреатит', 1.8973739),
 ('ревматоидный_артрит', 1.8945827),
 ('идиопатическая', 1.8940405),
 ('кожный_зуд', 1.8933771),
 ('скарлатина', 1.8892467),
 ('энтерит', 1.8886458),
 ('нейродермит', 1.8794886),
 ('сифилис', 1.8780508),
 ('корь', 1.8758836),
 ('простатит', 1.8717788),
 ('вирусная_инфекция', 1.8708802),
 ('конъюнктивит', 1.8705481),
 ('гангрена', 1.8687508),
 ('себорея', 1.8676116),
 ('склеродермия', 1.8665794),
 ('саркома', 1.8652024),
 ('кишечная_непроходимость', 1.8647914),
 ('псориаз', 1.8619876),
 ('эпилепсия', 1.8608676),
 ('стенокардия', 1.8591377),
 ('радикулит', 1.8584806),
 ('желтуха', 1.8573427),
 ('подагра', 1.8563939),
 ('системная_красная', 1.85